# Configuration and Usage

This notebook describes the basic concepts and configuration of Pulla.

## Basics

In [1]:
import os
from pprint import pprint
from qiskit import QuantumCircuit
from qiskit.compiler import transpile
from iqm.qiskit_iqm import IQMProvider
from iqm.pulla.utils_qiskit import qiskit_to_pulla

Now let's create a Pulla object.

A Pulla object is conceptually an **IQM quantum computer client for fetching calibration data and constructing a circuit-to-pulse compiler.**. It consists of:

- methods for fetching calibration sets from the server
- methods for fetching metadata about the QC from the server
- method for executing pulse-level instruction schedules (e.g. ones created by the compiler)

A Compiler object defines a **particular circuit-to-pulse compilation logic**. It consists of:

- single calibration set
- schedule builder based on the calibration set
- circuit compilation options
- information about the QC (chip topology, channel properties, etc.)
- compilation stages
- set of available circuit-level quantum operations ("native operations") (including user-defined operations)
- set of implementations for each native operation (including user-defined implementations)
- methods for manipulating the calibration, operations, and implementations

Pulla can construct a standard compiler equivalent to the one used by the server side (IQM Server). You can also construct a Compiler manually.

To create an instance of Pulla, you need to provide the URL of IQM Server. Upon successful initialization, some configuration data is printed (the verbosity of such messages will be controlled by a debug level value).

In [2]:
from iqm.pulla.pulla import Pulla

iqm_server_url = os.environ['PULLA_IQM_SERVER_URL']  # or set the URL directly here
os.environ["IQM_TOKEN"] = os.environ.get("IQM_TOKEN")  # or set the token directly here

p = Pulla(iqm_server_url)

You can access various things like channel properties of the connected station, chip topology, or the current calibration. The initial calibration is the default calibration of the connected station.

In [3]:
pprint(p.get_channel_properties()[0]['QB1__drive.awg'])

ChannelProperties(sample_rate=2000000000,
                  instruction_duration_granularity=8,
                  instruction_duration_min=8,
                  compatible_instructions=(),
                  is_iq=True,
                  is_virtual=False,
                  blocks_component=True)


In [4]:
p.get_chip_topology().get_coupler_for('QB1', 'QB3')

'TC-1-3'

Call `get_standard_compiler()` method to get an instance of `Compiler`. It will be pre-populated with the latest default calibration set and standard compilation stages. This standard compiler is the same one that IQM Server uses on the server side when processing circuits.

In [5]:
compiler = p.get_standard_compiler()

In [6]:
# get_calibration_set_values() returns the entire calibration set
# here we filter the data to get just the instrument settings of QB1

pprint({k: v for k, v in compiler.get_calibration_set_values().items() if k.startswith('controllers.QB1')})

{'controllers.QB1.drive.awg.center_frequency': 4500000000.0,
 'controllers.QB1.drive.awg.trigger_delay': 5.021e-07,
 'controllers.QB1.drive.frequency': 4348389108.22752,
 'controllers.QB1.flux.voltage': 0.48763612741829193}


`get_standard_compiler()` fetches the default calibration set from the server. This network request takes a few moments. You may want to create many Compiler instances without such delay.

It may also be possible that, due to human error, the default calibration set stored on the server is invalid (or incompatible with your version of Pulla or IQM Pulse). In that case `get_standard_compiler()` will fail.

To avoid this, you can pass the calibration set manually, and Pulla will construct the compiler with it instead of getting one from the server. If you want to reuse the calibration of an existing compiler, call `Compiler.get_calibration_set_values()`, e.g.:

```python
compiler_new = p.get_standard_compiler(calibration_set_values=compiler_old.get_calibration_set_values())
```

You can also get a specific calibration set from the server if you know its UUID by calling `Pulla.fetch_calibration_set_by_id()`, e.g.:

```python
specific_cal_set = p.fetch_calibration_set_values_by_id('fe026208-19aa-4906-93ab-06ba3c86100f')
compiler = p.get_standard_compiler(calibration_set_values=specific_cal_set)
```

The compiler initializes with the following default configuration options:

- `circuit_boundary_mode='all'`
- `measurement_mode='all'`
- `heralding_mode='none'`
- `dd_mode='none'`

You can change them by changing the corresponding attributes of `compiler.options`.

## Complex readout

For the `constant` implementation of the `measure` operation, the readout type is controlled by the `acquisition_type` parameter. By default, it's set to `"threshold"`. Let's change it to `"complex"`. The full key in the calibration set dictionary is `gates.measure.constant.QUBIT.acquisition_type`, where `QUBIT` is the physical qubit name. 

Note that we call `get_calibration_set_values()` to get a copy of the compiler's current calibration set, make changes to the copy, then replace the compiler's set with the copy. The compiler always contains a single calibration set only.

In [7]:
provider = IQMProvider(iqm_server_url)
backend = provider.get_backend()
shots = 10

qc = QuantumCircuit(3, 3)
qc.h(0)
qc.cx(0, 1)
qc.cx(0, 2)
qc.measure_all()

qc_transpiled = transpile(qc, backend=backend, layout_method='sabre', optimization_level=3)
circuits, compiler = qiskit_to_pulla(p, backend, qc_transpiled)

# change the acquisition type to 'complex' in the calibration set
updated_cal_set = compiler.get_calibration_set_values()
for qubit in backend.physical_qubits:
    updated_cal_set[f'gates.measure.constant.{qubit}.acquisition_type'] = 'complex'
compiler.set_calibration_set_values(updated_cal_set)

playlist, context = compiler.compile(circuits)
settings, context = compiler.build_settings(context, shots=shots)
job = p.submit_playlist(playlist, settings, context=context)
job.wait_for_completion()

# Pulla.submit_playlist() returns a SweepJob object.
# The measurements are obtained using SweepJob.result() after SweepJob.wait_for_completion().

print(f"Raw results:\n{job.result()}\n")

[06-17 14:44:00;I] Created job in queue with ID: f3917fe0-a1d3-4d0d-87dc-06300738839b
[06-17 14:44:00;I] Job link: http://varda.qc.iqm.fi/station/jobs/f3917fe0-a1d3-4d0d-87dc-06300738839b
[06-17 14:44:00;I] Waiting for the job to finish...
[06-17 14:44:01;I] Waiting for job ID: f3917fe0-a1d3-4d0d-87dc-06300738839b
[06-17 14:44:03;I] Sweep status: ready
Raw results:
[{'meas_3_1_2': [[(-0.004069618054976066+0.044584868128101036j)], [(-0.009010526090860367+0.04521688704689344j)], [(-0.005457488770286243+0.04069884149357677j)], [(-0.019717537785569828+0.04068584050362309j)], [(-0.014776446806887786+0.0407774516257147j)], [(-0.005077934470027685+0.043093412406742575j)], [(-0.01703992574165265+0.03919237102692326j)], [(-0.02089414958531658+0.03935748310138782j)], [(-0.004082951831320921+0.04183203347648184j)], [(-0.016026709278424582+0.043600302773217366j)]], 'meas_3_1_1': [[(0.003939125500619411-0.0028286848030984403j)], [(0.0017112398147583008-0.0028760674037039283j)], [(0.0047653886054952